In [1]:
import pickle
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
train_df = pd.read_csv("../../input/feedback-prize-english-language-learning/train.csv")
train_df.head(2)

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5


In [3]:
model_path_list = [
    '../../01_Baseline/exp/result/01_v1_24/oof_df.csv', # deberta-base
    '../../01_Baseline/exp/result/01_v1_25/oof_df.csv', # deberta-large
    '../../01_Baseline/exp/result/01_v1_27/oof_df.csv', # deberta-v3-base

    '../../01_Baseline/exp/result/01_v1_43/oof_df.csv', # deberta-v3-large
    '../../01_Baseline/exp/result/01_v1_44/oof_df.csv', # roberta-base
    '../../01_Baseline/exp/result/01_v1_45/oof_df.csv', # roberta-large
    '../../01_Baseline/exp/result/01_v1_47/oof_df.csv', # electra-base
    '../../01_Baseline/exp/result/01_v1_49/oof_df.csv', # deepset/roberta-base-squad2
    '../../01_Baseline/exp/result/01_v1_50/oof_df.csv', # distilroberta-base
    
    '../../01_Baseline/exp/result/01_v1_54/oof_df.csv', # electra-large
    '../../01_Baseline/exp/result/01_v1_55/oof_df.csv', # deepset/roberta-large-squad2
    '../../01_Baseline/exp/result/01_v1_56/oof_df.csv', # xlm-roberta-large
    '../../01_Baseline/exp/result/01_v1_59/oof_df.csv', # bert-large-cased-whole-word-masking
    '../../01_Baseline/exp/result/01_v1_60/oof_df.csv', # bert-large-cased
    '../../07_Embeddings/exp/result/07_v1_01/oof_df_svr_notebook_v03.csv', # svr
    
]

oof_df_list = [
    pd.read_csv(model_path) for model_path in model_path_list
]

for oof_df in oof_df_list:
    oof_df = train_df[['text_id']].merge(oof_df, how='left', on='text_id')
    
num_models = len(model_path_list)

TARGET_COLS = ['cohesion','syntax','vocabulary','phraseology','grammar','conventions']

preds = 0
for oof_df in oof_df_list:
    preds += oof_df[TARGET_COLS].values / num_models

oofs = []
for oof_df in oof_df_list:
    oofs.append(oof_df[TARGET_COLS].values)

In [4]:
oof_df = oof_df_list[0].copy()
for i,col in enumerate(TARGET_COLS):
    oof_df[col] = preds[:,i]

In [5]:
import numpy as np

def calc_metric(pred, gt):
    '''
    pred : (num_data, num_labels)
    gt : (num_data, num_labels)
    '''
    score = np.sqrt(np.mean((pred - gt)**2, axis=0))
    score = score.mean()
    return score

In [6]:
score = calc_metric(pred=oof_df[TARGET_COLS].values, gt=train_df[TARGET_COLS].values)
print('CV={:.4f}'.format(score))

CV=0.4489


In [7]:
ensemble_predictions=np.stack(oofs)

In [8]:
def ensemble_score(weights,return_pred=False):
    weights=np.array(weights)
    weights=weights.reshape(-1,1,1)/weights.sum()
    p=weights.reshape(-1,1,1)*ensemble_predictions
    p=p.sum(0)
    score=calc_metric(p, train_df[TARGET_COLS])
    if return_pred:
        return score,p
    else:
        return score   

In [9]:
from skopt import gp_minimize

results=gp_minimize(ensemble_score, np.array([[0.1,1] for i in range(len(ensemble_predictions))]),
                    verbose=True,random_state=2022)

Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.0060
Function value obtained: 0.4500
Current minimum: 0.4500
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.0034
Function value obtained: 0.4494
Current minimum: 0.4494
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 0.0027
Function value obtained: 0.4486
Current minimum: 0.4486
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 0.0026
Function value obtained: 0.4501
Current minimum: 0.4486
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 0.0027
Function value obtained: 0.4480
Current minimum: 0.4480
Iteration No: 6 started. Evaluating

Iteration No: 41 ended. Search finished for the next optimal point.
Time taken: 1.2553
Function value obtained: 0.4468
Current minimum: 0.4468
Iteration No: 42 started. Searching for the next optimal point.
Iteration No: 42 ended. Search finished for the next optimal point.
Time taken: 0.9545
Function value obtained: 0.4480
Current minimum: 0.4468
Iteration No: 43 started. Searching for the next optimal point.
Iteration No: 43 ended. Search finished for the next optimal point.
Time taken: 0.9929
Function value obtained: 0.4491
Current minimum: 0.4468
Iteration No: 44 started. Searching for the next optimal point.
Iteration No: 44 ended. Search finished for the next optimal point.
Time taken: 1.0206
Function value obtained: 0.4470
Current minimum: 0.4468
Iteration No: 45 started. Searching for the next optimal point.
Iteration No: 45 ended. Search finished for the next optimal point.
Time taken: 0.8503
Function value obtained: 0.4469
Current minimum: 0.4468
Iteration No: 46 started. Sea

Iteration No: 81 ended. Search finished for the next optimal point.
Time taken: 1.1698
Function value obtained: 0.4493
Current minimum: 0.4467
Iteration No: 82 started. Searching for the next optimal point.
Iteration No: 82 ended. Search finished for the next optimal point.
Time taken: 1.1880
Function value obtained: 0.4492
Current minimum: 0.4467
Iteration No: 83 started. Searching for the next optimal point.
Iteration No: 83 ended. Search finished for the next optimal point.
Time taken: 1.3077
Function value obtained: 0.4467
Current minimum: 0.4467
Iteration No: 84 started. Searching for the next optimal point.
Iteration No: 84 ended. Search finished for the next optimal point.
Time taken: 1.2799
Function value obtained: 0.4495
Current minimum: 0.4467
Iteration No: 85 started. Searching for the next optimal point.
Iteration No: 85 ended. Search finished for the next optimal point.
Time taken: 1.3493
Function value obtained: 0.4491
Current minimum: 0.4467
Iteration No: 86 started. Sea

In [10]:
best_weights=np.array(results['x'])/sum(results['x'])
print(best_weights)

[0.01514903 0.12219635 0.13667188 0.15149034 0.01514903 0.15149034
 0.01514903 0.01514903 0.01514903 0.01514903 0.11441798 0.05105052
 0.01514903 0.01514903 0.15149034]


In [11]:
# smaller is better for this metric
score,ensemble_pred=ensemble_score(best_weights,True)
score

0.44669140936081964

In [12]:
print('CV={:.4f}'.format(score))

CV=0.4467
